In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy.optimize import curve_fit
import glob
from astropy.table import Table, vstack
import pandas as pd
import statsmodels.api as sm
import pandas as pd
from astropy.table import Table, vstack
import glob as glob
from datetime import datetime
from collections import OrderedDict
import math

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397


In [ ]:
def reweight_gaug(data, zenith, is_data=False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        zenith = np.deg2rad(zenith)
        weights = np.ones(len(data))
        weights = data['mc_energy']**-0.7 * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/850))
        return weights / np.sum(weights)

def plot_reweighted_gaug(data, quantity, zenith=0, bins=50, range=(None, None), log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf), is_data = False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        weights = reweight_gaug(data, zenith)
    counts, bin_edges, patches = plt.hist(data[quantity], bins=bins, histtype='step', lw=2, log=log, label=label, weights=weights)
    y_limits = plt.gca().get_ylim()
    
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    
    plt.legend()
    plt.xlim(*range)
    
def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

def filter_dataframe(df, muon_efficiency_cut, size_outside_cut):
    return df[
        df['good_ring'] & 
        (df['muon_efficiency'] < muon_efficiency_cut) & 
        (df['size_outside'] < size_outside_cut)
    ]

def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_radius_cut(df, min_radius_cut, max_radius_cut):
    return df[
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut)
    ]

In [ ]:
# Process the first set of files
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_00025alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00025_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00046_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_00092alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0092_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0015_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_002alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_002_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

listdir = glob.glob('/Users/vdk/muons2024/psf_work/focus_offset_0/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_focus_offset0 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/data_quality_winter_2024/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_winter_data = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

In [ ]:
ring_completeness_cut = 0.9
ring_containment_cut = 0
min_radius_cut = 0
max_radius_cut = 2
ring_center_distance_cut = 0.65
impact_distance_cut = 6.5

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    # Calculate the 20% quantile for radial_stdev
    ring_width_quantile = df['radial_stdev'].quantile(1)
    
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_x'].abs() < center_distance_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut) &
        (df['radial_stdev'] < ring_width_quantile) 
    ]

df_sim_00025_cut = apply_cuts(df_sim_00025_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00025_cut: {len(df_sim_00025_cut)} ({calculate_survival_percentage(df_sim_00025_align, df_sim_00025_cut):.2f}%)")


df_sim_00046_cut = apply_cuts(df_sim_00046_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00046_cut: {len(df_sim_00046_cut)} ({calculate_survival_percentage(df_sim_00046_align, df_sim_00046_cut):.2f}%)")


df_sim_0092_cut = apply_cuts(df_sim_0092_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0092_cut: {len(df_sim_0092_cut)} ({calculate_survival_percentage(df_sim_0092_align, df_sim_0092_cut):.2f}%)")


df_sim_0015_cut = apply_cuts(df_sim_0015_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_align, df_sim_0015_cut):.2f}%)")

#df_sim_focus_offset0_cut = apply_cuts(df_sim_focus_offset0, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
#print(f"Events survived for df_sim_focus_offset0_cut: {len(df_sim_focus_offset0_cut)} ({calculate_survival_percentage(df_sim_focus_offset0, df_sim_focus_offset0_cut):.2f}%)")

#df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
#print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%)")

df_winter_2024_cut = apply_cuts(df_winter_data, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_winter_2024_cut: {len(df_winter_2024_cut)} ({calculate_survival_percentage(df_winter_data, df_winter_2024_cut):.2f}%)")

In [ ]:
min_radius_cut = 1.15
max_radius_cut = 1.23

bin_edges = max_radius_cut - min_radius_cut
bin_bins = np.linspace(min_radius_cut, max_radius_cut, 10)
bin_centers = (bin_bins[1:] + bin_bins[:-1]) / 2

df_sim_00025_radius_cut = apply_radius_cut(df_sim_00025_cut, min_radius_cut, max_radius_cut)
df_sim_00046_radius_cut = apply_radius_cut(df_sim_00046_cut, min_radius_cut, max_radius_cut)
df_sim_0092_radius_cut = apply_radius_cut(df_sim_0092_cut, min_radius_cut, max_radius_cut)
df_sim_0015_radius_cut = apply_radius_cut(df_sim_0015_cut, min_radius_cut, max_radius_cut)
df_sim_002_radius_cut = apply_radius_cut(df_sim_002_cut, min_radius_cut, max_radius_cut)
df_sim_focus_offset0_radius_cut = apply_radius_cut(df_sim_focus_offset0_cut, min_radius_cut, max_radius_cut)

df_winter_2024_radius_cut = apply_radius_cut(df_winter_2024_cut, min_radius_cut, max_radius_cut)
#df_data_cut_radius_cut = apply_radius_cut(df_data_cut, min_radius_cut, max_radius_cut)

print(f"2024 Winter Data ring radius min and max: {min(df_winter_2024_radius_cut['ring_radius']):.3f}, {max(df_winter_2024_radius_cut['ring_radius']):.3f}")
#print(f"All 2024 Data ring radius min and max: {min(df_data_cut_radius_cut['ring_radius']):.3f}, {max(df_data_cut_radius_cut['ring_radius']):.3f}")

print(f"For 0.0025 deg radius min and max = {min(df_sim_00025_radius_cut['ring_radius']):.3f} and {max(df_sim_00025_radius_cut['ring_radius']):.3f}")
print(f"For 0.0046 deg radius min and max = {min(df_sim_00046_radius_cut['ring_radius']):.3f} and {max(df_sim_00046_radius_cut['ring_radius']):.3f}")
print(f"For 0.0092 deg radius min and max = {min(df_sim_0092_radius_cut['ring_radius']):.3f} and {max(df_sim_0092_radius_cut['ring_radius']):.3f}")
print(f"For 0.015 deg radius min and max = {min(df_sim_0015_radius_cut['ring_radius']):.3f} and {max(df_sim_0015_radius_cut['ring_radius']):.3f}")
print(f"For 0.002 deg radius min and max = {min(df_sim_002_radius_cut['ring_radius']):.3f} and {max(df_sim_002_radius_cut['ring_radius']):.3f}")
print(f"For focus offset 0 deg radius min and max = {min(df_sim_focus_offset0_radius_cut['ring_radius']):.3f} and {max(df_sim_focus_offset0_radius_cut['ring_radius']):.3f}")

In [ ]:
df_winter_2024_radius_cut['radius_bin'] = pd.cut(df_winter_2024_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)

df_sim_00025_radius_cut['radius_bin'] = pd.cut(df_sim_00025_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_00046_radius_cut['radius_bin'] = pd.cut(df_sim_00046_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_0092_radius_cut['radius_bin'] = pd.cut(df_sim_0092_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_0015_radius_cut['radius_bin'] = pd.cut(df_sim_0015_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_002_radius_cut['radius_bin'] = pd.cut(df_sim_002_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_focus_offset0_radius_cut['radius_bin'] = pd.cut(df_sim_focus_offset0_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)

weights_00025 = reweight_gaug(df_sim_00025_radius_cut, 10.0)
weights_00046 = reweight_gaug(df_sim_00046_radius_cut, 10.0)
weights_0092 = reweight_gaug(df_sim_0092_radius_cut, 10.0)
weights_0015 = reweight_gaug(df_sim_0015_radius_cut, 10.0)
weights_winter = reweight_gaug(df_winter_2024_radius_cut, 10.0, True)    

# Ring width grouping by bins
data_radial_stdev_binned = df_winter_2024_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()
sim_00025_radial_stdev_binned = df_sim_00025_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()
sim_00046_radial_stdev_binned = df_sim_00046_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()
sim_0092_radial_stdev_binned = df_sim_0092_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()
sim_0015_radial_stdev_binned = df_sim_0015_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()
sim_002_radial_stdev_binned = df_sim_002_radius_cut.groupby('radius_bin')['radial_stdev'].agg(['mean', 'sum', 'count', 'std']).reset_index()

data_radial_stdev_binned['error'] = data_radial_stdev_binned['std'] / np.sqrt(data_radial_stdev_binned['count'])

sim_00025_radial_stdev_binned['error'] = sim_00025_radial_stdev_binned['std'] / np.sqrt(sim_00025_radial_stdev_binned['count'])
sim_00046_radial_stdev_binned['error'] = sim_00046_radial_stdev_binned['std'] / np.sqrt(sim_00046_radial_stdev_binned['count'])
sim_0092_radial_stdev_binned['error'] = sim_0092_radial_stdev_binned['std'] / np.sqrt(sim_0092_radial_stdev_binned['count'])
sim_0015_radial_stdev_binned['error'] = sim_0015_radial_stdev_binned['std'] / np.sqrt(sim_0015_radial_stdev_binned['count'])
sim_002_radial_stdev_binned['error'] = sim_002_radial_stdev_binned['std'] / np.sqrt(sim_002_radial_stdev_binned['count'])


In [ ]:
plt.figure(figsize=(10,7))
bin_bins = np.linspace(0,0.2,100)
# z,x,c = plt.hist(df_winter_2024_cut['ring_width'], bins=bin_bins, histtype='step', label = 'Ring Width')
z,x,c = plt.hist(df_sim_00046_radius_cut['radial_stdev'], bins=bin_bins, histtype='step', label ='0.0046', density=True, weights=weights_00046)
#z,x,c = plt.hist(df_sim_00025_radius_cut['radial_stdev'], bins=bin_bins, histtype='step', label ='0.0025', density=True, weights=weights_00025)
#z,x,c = plt.hist(df_sim_0092_radius_cut['radial_stdev'], bins=bin_bins, histtype='step', label ='0.0092', density=True, weights=weights_0092)
z,x,c = plt.hist(df_sim_0015_radius_cut['radial_stdev'], bins=bin_bins, histtype='step', label ='0.015', density=True, weights=weights_0015)
z,x,c = plt.hist(df_winter_2024_cut['radial_stdev'], bins=bin_bins, histtype='step', label ='Data winter 2024', density=True, weights=weights_winter, color = 'k', lw = 2)
plt.legend()

In [ ]:
plt.figure(figsize=(11, 7.5)) 

plt.errorbar(
    bin_centers,
    data_radial_stdev_binned['mean'] / bin_centers,
    yerr=data_radial_stdev_binned['error'] / bin_centers,
    fmt='x',
    label='Good Data Winter 2024',
    color='k',
    capsize=10,
    markersize=15
)

# plt.errorbar(
#     bin_centers,
#     all_data_ring_width_binned['mean'] / bin_centers,
#     yerr=all_data_ring_width_binned['error'] / bin_centers,
#     fmt='o',
#     label='All Data 2024',
#     color='black',
#     capsize=5
# )

plt.errorbar(
    bin_centers,
    sim_00025_width_binned['mean'] / bin_centers,
    yerr=sim_00025_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0025',
    color='purple',
    capsize=5,
)

plt.errorbar(
    bin_centers,
    sim_00046_width_binned['mean'] / bin_centers,
    yerr=sim_00046_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0046',
    color='orange',
    capsize=5
)


plt.errorbar(
    bin_centers,
    sim_0092_width_binned['mean'] / bin_centers,
    yerr=sim_0092_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0092',
    color='green',
    capsize=5
)


plt.errorbar(
    bin_centers,
    sim_0015_width_binned['mean'] / bin_centers,
    yerr=sim_0015_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.015',
    color='red',
    capsize=5
)

plt.errorbar(
    bin_centers,
    sim_002_width_binned['mean'] / bin_centers,
    yerr=sim_002_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.02',
    color='blue',
    capsize=5
)

# plt.errorbar(
#     bin_centers,
#     sim_focus_offset0_width_binned['mean'] / bin_centers,
#     yerr=sim_focus_offset0_width_binned['error'] / bin_centers,
#     fmt='o',
#     label='MC Focus Offset 0',
#     color='brown',
#     capsize=5
# )

plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.title('Mean Ring Width per Radius Bin with Statistical Errors')
plt.legend()
plt.ylim(0.01, 0.1)
plt.grid(alpha=0.6)
plt.tight_layout()  # Adjusts subplot params for better layout
plt.show()

In [ ]:
# Organize datasets into a list of dictionaries
datasets = [
    {'df': data_radial_stdev_binned, 'label': 'Data', 'color': 'gray'},
    #{'df': all_data_ring_width_binned, 'label': 'All Data 2024 year', 'color': 'black'},
    {'df': sim_00025_width_binned, 'label': 'MC 0.0025', 'color': 'purple'},
    {'df': sim_00046_width_binned, 'label': 'MC 0.0046', 'color': 'orange'},
    {'df': sim_0092_width_binned, 'label': 'MC 0.0092', 'color': 'green'},
    {'df': sim_0015_width_binned, 'label': 'MC 0.015', 'color': 'red'},
    {'df': sim_002_width_binned, 'label': 'MC 0.02', 'color': 'blue'},
    #{'df': sim_focus_offset0_width_binned, 'label': 'MC Focus Offset 0', 'color': 'brown'}
]

datasets = [
    #{'df': sim_00025_width_binned, 'label': 'MC 0.0025', 'color': 'purple'},
    {'df': sim_00046_width_binned, 'label': 'MC 0.0046', 'color': 'orange'},
    {'df': sim_0092_width_binned, 'label': 'MC 0.0092', 'color': 'green'},
    {'df': sim_0015_width_binned, 'label': 'MC 0.015', 'color': 'red'},
    #{'df': sim_002_width_binned, 'label': 'MC 0.02', 'color': 'blue'},
]

# Initialize the plot
plt.figure(figsize=(11, 7.5))

# Iterate over each dataset to plot error bars and regression lines
for dataset in datasets:
    df = dataset['df']
    label = dataset['label']
    color = dataset['color']
    x = bin_centers
    y = df['mean'] / x
    y_err = df['error'] / x

    plt.errorbar(
        x,
        y,
        yerr=y_err,
        fmt='o',
        label=label,
        color=color,
        capsize=5
    )
    
    slope, intercept = np.polyfit(x, y, 1)
    y_fit = slope * x + intercept
    plt.plot(x, y_fit, color=color, linestyle='--')

plt.errorbar(
    bin_centers,
    data_radial_stdev_binned['mean']/bin_centers,
    yerr=data_radial_stdev_binned['error']/bin_centers,
    fmt='x',
    label='Good quality data 2024 winter',
    color='k',
    capsize=10
)

slope, intercept = np.polyfit(x, data_radial_stdev_binned['mean']/bin_centers, 1)
y_fit = slope * bin_centers + intercept
plt.plot(bin_centers, y_fit, color='k', linestyle='-', lw = 2)

# Adjust legend to avoid duplicate entries
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

# Customize the plot
plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.ylim(0.01, 0.06)
plt.title(f'min/max radius cut = {min_radius_cut}/{max_radius_cut}, completeness = {ring_completeness_cut} center distance cut < {ring_center_distance_cut}, impact distance cut < {impact_distance_cut}')
plt.grid(alpha=0.3)
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
def cherenkov_angle(muon_energy_eV, refractive_index=1.000293):
    """
    Calculate the Cherenkov angle for a given muon energy and refractive index.

    Parameters:
    - muon_energy_eV (float): The energy of the muon in electron volts (eV).
    - refractive_index (float, optional): The refractive index of the medium. Default is 1.000293.

    Returns:
    - theta_C_degrees (float): The Cherenkov angle in degrees.

    Raises:
    - None

    """
    # Constants
    c = 299792458  # speed of light in m/s
    muon_rest_mass_MeV = 105.66  # rest mass of muon in MeV
    eV_to_MeV = 1e-6  # conversion factor from eV to MeV
    
    # Convert muon energy from eV to MeV
    muon_energy_MeV = muon_energy_eV * eV_to_MeV
    
    # Check if the energy is sufficient for Cherenkov radiation
    if muon_energy_MeV <= muon_rest_mass_MeV:
        return "Energy too low for Cherenkov radiation."
    
    # Calculate the momentum p
    p = math.sqrt((muon_energy_MeV)**2 - (muon_rest_mass_MeV)**2)
    
    # Calculate beta
    beta = p / muon_energy_MeV
    
    # Check if Cherenkov condition is met
    if beta * refractive_index <= 1:
        return "Muon velocity too low for Cherenkov radiation in this medium."
    
    # Calculate Cherenkov angle
    cos_theta_C = 1 / (beta * refractive_index)
    theta_C = math.acos(cos_theta_C)
    
    # Convert angle from radians to degrees
    theta_C_degrees = math.degrees(theta_C)
    
    return theta_C_degrees

def plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points, refractive_index=1.000293, plot_option=False):
    """
    Plots the Cherenkov angle distribution as a function of muon energy.

    Parameters:
    - min_energy_eV (float): The minimum energy in electron volts (eV) for the muon.
    - max_energy_eV (float): The maximum energy in electron volts (eV) for the muon.
    - num_points (int): The number of points to generate between the minimum and maximum energy.
    - refractive_index (float, optional): The refractive index of the medium. Default is 1.000293.
    - plot_option (bool, optional): Whether to plot the distribution. Default is False.

    Returns:
    - energies_GeV (numpy.ndarray): An array of muon energies in gigaelectron volts (GeV).
    - angles (list): A list of Cherenkov angles in degrees corresponding to the muon energies.

    Example usage:
    >>> min_energy_eV = 1e6
    >>> max_energy_eV = 1e9
    >>> num_points = 100
    >>> refractive_index = 1.000293
    >>> plot_option = True
    >>> energies_GeV, angles = plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points, refractive_index, plot_option)
    """

    energies_eV = np.logspace(np.log10(min_energy_eV), np.log10(max_energy_eV), num_points)
    energies_GeV = energies_eV * 1e-9  # Convert eV to GeV
    angles = []

    for energy in energies_eV:
        angle = cherenkov_angle(energy, refractive_index)
        if angle is not None:
            angles.append(angle)
        else:
            angles.append(0)  # Append 0 for energies not producing Cherenkov radiation

    if plot_option:
        plt.plot(energies_GeV, angles, label='Cherenkov Angle')
        plt.xscale('log')
        plt.xlabel('Muon Energy (GeV)')
        plt.ylabel('Cherenkov Angle (degrees)')
        plt.title('Cherenkov Angle Distribution as a Function of Muon Energy')
        plt.legend()
        plt.show()

    return energies_GeV, angles